In [1]:
# Download files using wget or urllib
import urllib.request
import zipfile
import os

# Function to download a file
def download_file(url, filename):
    urllib.request.urlretrieve(url, filename)

# Download and unzip files
# download_file("https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_1_states_provinces.zip", "ne_10m_admin_1_states_provinces.zip")
# with zipfile.ZipFile("ne_10m_admin_1_states_provinces.zip", 'r') as zip_ref:
#     zip_ref.extractall(".")

download_file("https://github.com/diegormsouza/aemet_eumetsat_uruguay_2023/raw/main/ancillary/cpt_color_tables.zip", "cpt_color_tables.zip")
with zipfile.ZipFile("cpt_color_tables.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

download_file("https://raw.githubusercontent.com/diegormsouza/aemet_eumetsat_uruguay_2023/main/ancillary/utilities.py", "utilities.py")
download_file("https://raw.githubusercontent.com/diegormsouza/oceanography_python_may_2022/main/utilities_ocean.py", "utilities_ocean.py")
download_file("https://raw.githubusercontent.com/diegormsouza/aemet_eumetsat_uruguay_2023/main/ancillary/abi_l2_nc.py", "abi_l2_nc.py")
download_file("https://raw.githubusercontent.com/diegormsouza/aemet_eumetsat_uruguay_2023/main/ancillary/test_abi_l2_nc.py", "test_abi_l2_nc.py")

# Copy files to appropriate directories (adjust paths for Windows)
import shutil

shutil.copy("abi_l2_nc.py", "D:/RODRIGO/IntradayForecasting/Condashboard/Lib/site-packages/satpy/readers/abi_l2_nc.py")  # Adjust path
shutil.copy("test_abi_l2_nc.py", "D:/RODRIGO/IntradayForecasting/Condashboard/Lib/site-packages/satpy/tests/reader_tests/test_abi_l2_nc.py")  # Adjust path

# Create directories
os.makedirs("sample-data", exist_ok=True)
os.makedirs("output", exist_ok=True)

In [ ]:
###############################################################################
# LICENSE
#Copyright (C) 2018 - INPE - NATIONAL INSTITUTE FOR SPACE RESEARCH
#This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.
#This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.
#You should have received a copy of the GNU General Public License along with this program. If not, see http://www.gnu.org/licenses/.
###############################################################################
#======================================================================================================
# GNC-A Blog Python Tutorial: Derived Motion Winds
#======================================================================================================

# Required libraries ==================================================================================
import matplotlib.pyplot as plt                         # Import the Matplotlib package
from mpl_toolkits.basemap import Basemap                # Import the Basemap toolkit
import numpy as np                                      # Import the Numpy package
from remap_g16 import remap                                 # Import the Remap function
from cpt_convert import loadCPT                         # Import the CPT convert function
from matplotlib.colors import LinearSegmentedColormap   # Linear interpolation for color maps
from datetime import datetime                           # Library to convert julian day to dd-mm-yyyy
from netCDF4 import Dataset                             # Import the NetCDF Python interface
import os                                # Miscellaneous operating system interfaces
import numpy as np                       # Import the Numpy package
import colorsys                          # To make convertion of colormaps
import matplotlib.colors as mcolors
import cartopy, cartopy.crs as ccrs  # Plot maps
import boto3                             # Amazon Web Services (AWS) SDK for Python
from botocore import UNSIGNED            # boto3 config
from botocore.config import Config       # boto3 config
import math                              # Mathematical functions
#from datetime import datetime            # Basic Dates and time types
import datetime
from osgeo import osr                    # Python bindings for GDAL
from osgeo import gdal                   # Python bindings for GDAL

#======================================================================================================

In [2]:
def download_CMI(yyyymmddhhmn, band, path_dest, product_name):

  os.makedirs(path_dest, exist_ok=True)

  year = datetime.datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
  day_of_year = datetime.datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%j')
  hour = datetime.datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
  min = datetime.datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')

  # AMAZON repository information
  # https://noaa-goes16.s3.amazonaws.com/index.html
  bucket_name = 'noaa-goes16'
  #product_name = 'ABI-L2-CMIPF'

  # Initializes the S3 client
  s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
  #-----------------------------------------------------------------------------------------------------------
  # File structure
  prefix = f'{product_name}/{year}/{day_of_year}/{hour}/OR_{product_name}-M6C{int(band):02.0f}_G16_s{year}{day_of_year}{hour}{min}'

  # Seach for the file on the server
  s3_result = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter = "/")

  #-----------------------------------------------------------------------------------------------------------
  # Check if there are files available
  if 'Contents' not in s3_result:
    # There are no files
    print(f'No files found for the date: {yyyymmddhhmn}, Band-{band}')
    return -1
  else:
    # There are files
    for obj in s3_result['Contents']:
      key = obj['Key']
      # Print the file name
      file_name = key.split('/')[-1].split('.')[0]

      # Download the file
      if os.path.exists(f'{path_dest}/{file_name}.nc'):
        print(f'File {path_dest}/{file_name}.nc exists')
      else:
        print(f'Downloading file {path_dest}/{file_name}.nc')
        s3_client.download_file(bucket_name, key, f'{path_dest}/{file_name}.nc')
  return f'{file_name}'

def get_info(path):
    #==================================================================================================
    # Getting Information From the File Name
    #==================================================================================================
    # Search for the Scan start in the file name
    Start = (path[path.find("_s")+2:path.find("_e")])
    # Converting from julian day to dd-mm-yyyy
    year = int(Start[0:4])
    dayjulian = int(Start[4:7]) - 1 # Subtract 1 because the year starts at "0"
    dayconventional = datetime.datetime(year,1,1) + datetime.timedelta(dayjulian) # Convert from julian to conventional
    date = dayconventional.strftime('%d-%b-%Y')  # Format the date according to the strftime directives
    time = Start [7:9] + ":" + Start [9:11] + ":" + Start [11:13] + " UTC" # Time of the Start of the Scan
    # Date as string
    date_save = dayconventional.strftime('%Y%m%d')
    # Time (UTC) as string
    time_save = Start [7:9] + Start [9:11]

    #==================================================================================================
    # Detect the product type
    #==================================================================================================
    start_index = path.find("L2-") + 3
    end_index_m3 = path.find("-M3")
    end_index_m4 = path.find("-M4")
    end_index_m6 = path.find("-M6")
    # Choose the minimum index value that is not -1
    end_index = min(index for index in [end_index_m3, end_index_m4, end_index_m6] if index != -1)

    product = path[start_index:end_index]
    print(product)

    # CMIPF - Cloud and Moisture Imagery: 'CMI'
    if (product == "CMIPF") or (product == "CMIPC") or (product == "CMIPM"):
        variable = 'CMI'
        vmin = -50
        vmax = 50
        cmap = "jet"

    # ACHAF - Cloud Top Height: 'HT'
    elif product == "ACHAF":
        variable = 'HT'
        vmin = 0
        vmax = 15000
        cmap = "rainbow"

    # ACHTF - Cloud Top Temperature: 'TEMP'
    elif product == "ACHTF":
        variable = 'TEMP'
        vmin = 180
        vmax = 300
        cmap = "jet"

    # ACMF - Clear Sky Masks: 'BCM'
    elif product == "ACMF":
        variable = 'BCM'
        vmin = 0
        vmax = 1
        cmap = "gray"

    # ACTPF - Cloud Top Phase: 'Phase'
    elif product == "ACTPF":
        variable = 'Phase'
        vmin = 0
        vmax = 5
        cmap = "jet"

    # ADPF - Aerosol Detection: 'Smoke'
    elif product == "ADPF":
        variable = 'Smoke'
        vmin = 0
        vmax = 255
        cmap = "jet"

        #variable = 'Dust'
        #vmin = 0
        #vmax = 255
        #cmap = "jet"

    # AODF - Aerosol Optical Depth: 'AOD'
    elif product == "AODF":
        variable = 'AOD'
        vmin = 0
        vmax = 2
        cmap = "rainbow"

    # CODF - Cloud Optical Depth: 'COD'
    elif product == "CODF":
        variable = 'COD'
        vmin = 0
        vmax = 100
        cmap = "jet"

    # CPSF - Cloud Particle Size: 'PSD'
    elif product == "CPSF":
        variable = 'PSD'
        vmin = 0
        vmax = 80
        cmap = "rainbow"

    # CTPF - Cloud Top Pressure: 'PRES'
    elif product == "CTPF":
        variable = 'PRES'
        vmin = 0
        vmax = 1100
        cmap = "rainbow"

    # DMWF - Derived Motion Winds: 'pressure','temperature', 'wind_direction', 'wind_speed'
    elif product == "DMWF":
        variable = 'pressure'
        #variable = 'temperature'
        #variable = 'wind_direction'
        #variable = 'wind_speed'

    # DSIF - Derived Stability Indices: 'CAPE', 'KI', 'LI', 'SI', 'TT'
    elif product == "DSIF":
        variable = 'CAPE'
        vmin = 0
        vmax = 1000
        cmap = "jet"

        #variable = 'KI'
        #vmin = -50
        #vmax = 50
        #cmap = "jet"

        #variable = 'LI'
        #vmin = -10
        #vmax = 30
        #cmap = "jet"

        #variable = 'SI'
        #vmin = -10
        #vmax = 25
        #cmap = "jet"

        #variable = 'TT'
        #vmin = -10
        #vmax = 60
        #cmap = "jet"

    # DSRF - Downward Shortwave Radiation: 'DSR'
    #elif product == "DSRF":
        #    variable = 'DSR'

    # FDCF - Fire-Hot Spot Characterization: 'Area', 'Mask', 'Power', 'Temp'
    elif product == "FDCF":
        #variable = 'Area'

        variable = 'Mask'
        vmin = 0
        vmax = 255
        cmap = "jet"

        #variable = 'Power'
        #variable = 'Temp'

    # FSCF - Snow Cover: 'FSC'
    elif product == "FSCF":
        variable = 'FSC'
        vmin = 0
        vmax = 1
        cmap = "jet"

    # LSTF - Land Surface (Skin) Temperature: 'LST'
    elif product == "LSTF":
        variable = 'LST'
        vmin = 213
        vmax = 330
        cmap = "jet"

    # RRQPEF - Rainfall Rate - Quantitative Prediction Estimate: 'RRQPE'
    elif product == "RRQPEF":
        variable = 'RRQPE'
        vmin = 0
        vmax = 50
        cmap = "jet"

    # RSR - Reflected Shortwave Radiation: 'RSR'
    #elif product == "RSRF":
        #    variable = 'RSR'

    # SSTF - Sea Surface (Skin) Temperature: 'SST'
    elif product == "SSTF":
        variable = 'SST'
        vmin = 268
        vmax = 308
        cmap = "jet"

    # TPWF - Total Precipitable Water: 'TPW'
    elif product == "TPWF":
        variable = 'TPW'
        vmin = 0
        vmax = 60
        cmap = "jet"

    # VAAF - Volcanic Ash: 'VAH', 'VAML'
    elif product == "VAAF":
        #variable = 'VAH'
        #vmin = 0
        #vmax = 20000
        #cmap = "jet"

        variable = 'VAML'
        vmin = 0
        vmax = 100
        cmap = "jet"

    return product, variable, vmin, vmax, cmap, date_save, time_save

#======================================================================================================
# Remap the GOES-16 file (Returns the remapped Grid)
#======================================================================================================
def remap_g16(path, extent, resolution):
    # Open the file using the NetCDF4 library
    nc = Dataset(path)

    # Get the latitude and longitude image bounds
    geo_extent = nc.variables['geospatial_lat_lon_extent']
    min_lon = float(geo_extent.geospatial_westbound_longitude)
    max_lon = float(geo_extent.geospatial_eastbound_longitude)
    min_lat = float(geo_extent.geospatial_southbound_latitude)
    max_lat = float(geo_extent.geospatial_northbound_latitude)

    # Calculate the image extent required for the reprojection
    H = nc.variables['goes_imager_projection'].perspective_point_height
    x1 = nc.variables['x_image_bounds'][0] * H
    x2 = nc.variables['x_image_bounds'][1] * H
    y1 = nc.variables['y_image_bounds'][1] * H
    y2 = nc.variables['y_image_bounds'][0] * H

    # Close the NetCDF file after getting the data
    nc.close()

    # Call the reprojection funcion
    grid = remap(path, variable, extent, resolution, x1, y1, x2, y2)

    return grid

def plot_g16(data, extent, product, variable, date_save, time_save):

    if (variable == "Dust") or (variable == "Smoke") or (variable == "TPW") or (variable == "PRES") or  (variable == "HT") or \
    (variable == "TEMP") or (variable == "AOD") or (variable == "COD") or (variable == "PSD") or  (variable == "CAPE") or  (variable == "KI") or \
    (variable == "LI") or (variable == "SI") or (variable == "TT") or (variable == "FSC") or  (variable == "RRQPE") or (variable == "VAML") or (variable == "VAH"):
        data[data == max(data[0])] = np.nan
        data[data == min(data[0])] = np.nan

    if (variable == "SST"):
        data[data == max(data[0])] = np.nan
        data[data == min(data[0])] = np.nan

        # Call the reprojection funcion again to get only the valid SST pixels
        grid = remap(path, "DQF", extent, resolution, x1, y1, x2, y2)
        data_DQF = grid.ReadAsArray()
        # If the Quality Flag is not 0, set as NaN
        data[data_DQF != 0] = np.nan

    if (variable == "Mask"):
        data[data == -99] = np.nan
        data[data == 40] = np.nan
        data[data == 50] = np.nan
        data[data == 60] = np.nan
        data[data == 150] = np.nan
        data[data == max(data[0])] = np.nan
        data[data == min(data[0])] = np.nan

    if (variable == "BCM"):
        data[data == 255] = np.nan
        data[data == 0] = np.nan

    if (variable == "Phase"):
        data[data >= 5] = np.nan
        data[data == 0] = np.nan

    if (variable == "LST"):
        data[data >= 335] = np.nan
        data[data <= 200] = np.nan

    if (variable == "CMI"):
        #data[data == max(data[0])] = np.nan
        #data[data == min(data[0])] = np.nan

        # Getting information from the file name ==============================================================
        # Search for the GOES-16 channel in the file name
        index_m3c = path.find("M3C")
        index_m4c = path.find("M4C")
        index_m6c = path.find("M6C")

        # Choose the minimum index value that is not -1
        start_index = min(index for index in [index_m3c, index_m4c, index_m6c] if index != -1) + 3

        end_index = path.find("_G16")
        band = int(path[start_index:end_index])

        # Create a GOES-16 Bands string array
        Wavelenghts = ['[]','[0.47 μm]','[0.64 μm]','[0.865 μm]','[1.378 μm]','[1.61 μm]','[2.25 μm]','[3.90 μm]','[6.19 μm]','[6.95 μm]','[7.34 μm]','[8.50 μm]','[9.61 μm]','[10.35 μm]','[11.20 μm]','[12.30 μm]','[13.30 μm]']
        # Get the unit based on the channel. If channels 1 trough 6 is Albedo. If channels 7 to 16 is BT.
        if Band <= 6:
            Unit = "Reflectance"
            data = data
        else:
            Unit = "Brightness Temperature [°C]"
            # If it is an IR channel subtract 273.15 to convert to ° Celsius
            data = data - 273.15
            # Make pixels outside the footprint invisible
            data[data <= -180] = np.nan

    # Choose a title for the plot
    #Title = " GOES-16 ABI CMI Band " + str(Band) + "       " +  Wavelenghts[int(Band)] + "       " + Unit + "       " + date + "       " + time
    # Insert the institution name
    #Institution = "GNC-A Blog"

    #==================================================================================================
    # Plot the Data
    #==================================================================================================

    if (variable == "CMI"):
        if Band <= 6:
            # Converts a CPT file to be used in Python
            cpt = loadCPT('Square Root Visible Enhancement.cpt')
            # Makes a linear interpolation
            cpt_convert = LinearSegmentedColormap('cpt', cpt)
            # Plot the GOES-16 channel with the converted CPT colors (you may alter the min and max to match your preference)
            bmap.imshow(data, origin='upper', cmap=cpt_convert, vmin=0, vmax=1)
            # Insert the colorbar at the bottom
        elif Band == 7:
            # Converts a CPT file to be used in Python
            cpt = loadCPT('SVGAIR2_TEMP.cpt')
            # Makes a linear interpolation
            cpt_convert = LinearSegmentedColormap('cpt', cpt)
            # Plot the GOES-16 channel with the converted CPT colors (you may alter the min and max to match your preference)
            bmap.imshow(data, origin='upper', cmap=cpt_convert, vmin=-112.15, vmax=56.85)
            # Insert the colorbar at the bottom
        elif Band > 7 and Band < 11:
            # Converts a CPT file to be used in Python
            cpt = loadCPT('SVGAWVX_TEMP.cpt')
            # Makes a linear interpolation
            cpt_convert = LinearSegmentedColormap('cpt', cpt)
            # Plot the GOES-16 channel with the converted CPT colors (you may alter the min and max to match your preference)
            bmap.imshow(data, origin='upper', cmap=cpt_convert, vmin=-112.15, vmax=56.85)
            # Insert the colorbar at the bottom
        elif Band > 10:
            # Converts a CPT file to be used in Python
            cpt = loadCPT('IR4AVHRR6.cpt')
            # Makes a linear interpolation
            cpt_convert = LinearSegmentedColormap('cpt', cpt)
            # Plot the GOES-16 channel with the converted CPT colors (you may alter the min and max to match your preference)
            #bmap.imshow(data, origin='upper', cmap='gray', vmin=-103, vmax=84)
            bmap.imshow(data, origin='upper', cmap='gray_r', vmin=-70, vmax=40)
            # Insert the colorbar at the bottom
    else:
        bmap.imshow(data, origin='upper', cmap=cmap, vmin=vmin, vmax=vmax)

    #cb = bmap.colorbar(location='bottom', size = '1%', pad = '-1.0%')
    #cb.outline.set_visible(False)                              # Remove the colorbar outline
    #cb.ax.tick_params(width = 0)                               # Remove the colorbar ticks
    #cb.ax.xaxis.set_tick_params(pad=-17)                     # Put the colobar labels inside the colorbar
    #cb.ax.tick_params(axis='x', colors='black', labelsize=12)  # Change the color and size of the colorbar labels

    # Save the result
    # plt.savefig('E:\\VLAB\\Python\\Output\\G16_' + product + '_' + variable + '_' + date_save + time_save + '.png', dpi=DPI, pad_inches=0)


In [ ]:
path_dest = 'content/Samples/'


yyyymmddhhmn = "202404220700"
Band = 14
file1 = download_CMI(yyyymmddhhmn, Band, path_dest, 'ABI-L2-CMIPF')
file2 = download_CMI(yyyymmddhhmn, Band, path_dest, 'ABI-L2-DMWF')

# Load the Data =======================================================================================
# Path to the GOES-16 image file
path = 'content/Samples/' + file1 + '.nc'
# Path to the GOES-16 derived motion winds and file reading:
path_dmwf = 'content/Samples/' + file2 + '.nc'

product, variable, vmin, vmax, cmap, date_save, time_save = get_info(path)

# South America
# extent = [-90.0, -60.0, -30.0, 12.0]

x_lat = 50
x_lon = 50
# extent = [-73.8, 9.78, -73.6, 9.80]
# extent = [-73.722451, 9.789103, -73.722451, 9.789103]
extent = [-73.722451-x_lon, 9.789103-x_lat, -73.722451+x_lon, 9.789103+x_lat]
# Choose the image resolution (the higher the number the faster the processing is)
resolution = 2

# Call the remap function
grid = remap_g16(path, extent, resolution)
# Read the data returned by the function
data_im = grid.ReadAsArray()
# Convert from int16 to uint16
data_im = data_im.astype(np.float64)









# Getting information from the file name ==============================================================
# Search for the Scan start in the file name
Start = (path[path.find("_s")+2:path.find("_e")])  # 20180831730455

# Search for the GOES-16 channel in the file name
# Band = int((path[path.find("M3C" or "M4C" or "M6C")+3:path.find("_G16")]))
# Create a GOES-16 Bands string array
Wavelenghts = ['[]','[0.47 μm]','[0.64 μm]','[0.865 μm]','[1.378 μm]','[1.61 μm]','[2.25 μm]','[3.90 μm]','[6.19 μm]','[6.95 μm]','[7.34 μm]','[8.50 μm]','[9.61 μm]','[10.35 μm]','[11.20 μm]','[12.30 μm]','[13.30 μm]']

# Converting from julian day to dd-mm-yyyy
year = int(Start[0:4])
dayjulian = int(Start[4:7]) - 1 # Subtract 1 because the year starts at "0"
dayconventional = datetime.datetime(year,1,1) + datetime.timedelta(dayjulian) # Convert from julian to conventional
date = dayconventional.strftime('%d-%b-%Y')  # Format the date according to the strftime directives

time = Start [7:9] + ":" + Start [9:11] + ":" + Start [11:13] + " UTC" # Time of the Start of the Scan

# Get the unit based on the channel. If channels 1 trough 6 is Albedo. If channels 7 to 16 is BT.
if Band <= 6:
    Unit = "Reflectance"
else:
    Unit = "Brightness Temperature [°C]"

# Choose a title for the plot
Title = " GOES-16 ABI CMI Band " + str(Band) + "       " +  Wavelenghts[int(Band)] + "       " + Unit + "       " + date + "       " + time
# Insert the institution name
Institution = "GNC-A Blog"
# =====================================================================================================

# Open the file using the NetCDF4 library
nc = Dataset(path)

# Get the latitude and longitude image bounds
geo_extent = nc.variables['geospatial_lat_lon_extent']
min_lon = float(geo_extent.geospatial_westbound_longitude)
max_lon = float(geo_extent.geospatial_eastbound_longitude)
min_lat = float(geo_extent.geospatial_southbound_latitude)
max_lat = float(geo_extent.geospatial_northbound_latitude)

# Full Disk
#extent = [min_lon, min_lat, max_lon, max_lat]

# South America
# extent = [-90.0, -60.0, -30.0, 12.0]

# x_lat = 50
# x_lon = 50
# extent = [-73.8, 9.78, -73.6, 9.80]
# extent = [-73.722451, 9.789103, -73.722451, 9.789103]
# extent = [-73.722451-x_lon, 9.789103-x_lat, -73.722451+x_lon, 9.789103+x_lat]

# Brazil
#extent = [-80.0, -40.0, -30.0, 10.0]

# Choose the image resolution (the higher the number the faster the processing is)
resolution = 2

# Calculate the image extent required for the reprojection
H = nc.variables['goes_imager_projection'].perspective_point_height
x1 = nc.variables['x_image_bounds'][0] * H
x2 = nc.variables['x_image_bounds'][1] * H
y1 = nc.variables['y_image_bounds'][1] * H
y2 = nc.variables['y_image_bounds'][0] * H

# Call the reprojection funcion
grid = remap(path, "DQF" ,extent, resolution, x1, y1, x2, y2)

# Read the data returned by the function
if Band <= 6:
    data = grid.ReadAsArray()
else:
    # If it is an IR channel subtract 273.15 to convert to ° Celsius
    data = grid.ReadAsArray() - 273.15
    # Make pixels outside the footprint invisible
    data[data <= -180] = np.nan

#======================================================================================================

# Define the size of the saved picture=================================================================
#print (data.shape)
DPI = 150
fig = plt.figure(figsize=(data.shape[1]/float(DPI), data.shape[0]/float(DPI)), frameon=False, dpi=DPI)
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
ax = plt.axis('off')
#======================================================================================================

# Plot the Data =======================================================================================
# Create the basemap reference for the Rectangular Projection
bmap = Basemap(llcrnrlon=extent[0], llcrnrlat=extent[1], urcrnrlon=extent[2], urcrnrlat=extent[3], epsg=4326)

if Band <= 6:
    # Converts a CPT file to be used in Python
    cpt = loadCPT('Square Root Visible Enhancement.cpt')
    # Makes a linear interpolation
    cpt_convert = LinearSegmentedColormap('cpt', cpt)
    # Plot the GOES-16 channel with the converted CPT colors (you may alter the min and max to match your preference)
    bmap.imshow(data, origin='upper', cmap=cpt_convert, vmin=0, vmax=1, alpha = 1.0)
    # Insert the colorbar at the bottom
elif Band == 7:
    # Converts a CPT file to be used in Python
    cpt = loadCPT('SVGAIR2_TEMP.cpt')
    # Makes a linear interpolation
    cpt_convert = LinearSegmentedColormap('cpt', cpt)
    # Plot the GOES-16 channel with the converted CPT colors (you may alter the min and max to match your preference)
    #bmap.imshow(data, origin='upper', cmap=cpt_convert, vmin=-112.15, vmax=56.85, alpha = 1.0)
    bmap.imshow(data, origin='upper', cmap='gray_r', vmin=-93.15, vmax=46.85, alpha = 1.0)
    # Insert the colorbar at the bottom
elif Band > 7 and Band < 11:
    # Converts a CPT file to be used in Python
    cpt = loadCPT('Colortables/SVGAWVX_TEMP.cpt')
    # Makes a linear interpolation
    cpt_convert = LinearSegmentedColormap('cpt', cpt)
    # Plot the GOES-16 channel with the converted CPT colors (you may alter the min and max to match your preference)
    #bmap.imshow(data, origin='upper', cmap=cpt_convert, vmin=-112.15, vmax=56.85, alpha = 1.0)
    bmap.imshow(data, origin='upper', cmap='gray_r', vmin=-93.15, vmax=46.85, alpha = 1.0)
    # Insert the colorbar at the bottom
elif Band > 10:
    # Converts a CPT file to be used in Python
    cpt = loadCPT('Colortables/IR4AVHRR6.cpt')
    # Makes a linear interpolation
    cpt_convert = LinearSegmentedColormap('cpt', cpt)
    # Plot the GOES-16 channel with the converted CPT colors (you may alter the min and max to match your preference)
    #bmap.imshow(data, origin='upper', cmap=cpt_convert, vmin=-103, vmax=84, alpha = 1.0)
    bmap.imshow(data, origin='upper', cmap='gray_r', vmin=-93.15, vmax=46.85, alpha = 1.0)
    #bmap.imshow(data, origin='upper', cmap='gray_r', vmin=-93.15, vmax=86.85, alpha = 1.0)

# Date as string
date_save = dayconventional.strftime('%Y%m%d')
# Time (UTC) as string
time_save = Start [7:9] + Start [9:11]

# Required libraries ==========================================================
import matplotlib.pyplot as plt          # Import the Matplotlib package
from netCDF4 import Dataset              # Import the NetCDF Python interface
import numpy as np                       # Import the Numpy package
from mpl_toolkits.basemap import Basemap # Import the Basemap toolkit
import math                              # Import the Math package
#import time as t                         # Import the Time package

# Opening the NetCDF
nc = Dataset(path_dmwf)

# Defining the visualization region: ==========================================
#South America
#extent = [-90.0, -60.0, -30.0, 12.0]

# GOES-16 Full Extent
#extent = [-156, -81, 6, 81]

# Read the required variables: ================================================
pressure = nc.variables['pressure'][:]
temperature = nc.variables['temperature'][:]
wind_direction = nc.variables['wind_direction'][:]
wind_speed = nc.variables['wind_speed'][:]
lats = nc.variables['lat'][:]
lons = nc.variables['lon'][:]

# Selecting data only from the region of interest: ============================
# Detect Latitude lower and upper index, according to the selected extent:
latli = np.argmin( np.abs( lats - extent[1] ) ) # Lower index
latui = np.argmin( np.abs( lats - extent[3] ) ) # Upper index

# Detect the Longitude index:
# Store the indexes where the lons are between the selected extent:
lon_ind = np.where(( lons >= extent[0]) & (lons <= extent[2] ))[0]
# Eliminate the lon indexes where we don't have the lat indexes:
lon_ind = lon_ind[(lon_ind >= latui) & (lon_ind <= latli)]

# Create the variables lists ==================================================
pressure_a = []
temperature_a = []
wind_direction_a = []
wind_speed_a = []
lats_a = []
lons_a = []

# For each item, append the values to the respective variables ================
for item in lon_ind:
    lons_a.append(lons[item])
    lats_a.append(lats[item])
    pressure_a.append(pressure[item])
    temperature_a.append(temperature[item])
    wind_direction_a.append(wind_direction[item])
    wind_speed_a.append(wind_speed[item])

# Read the variables as numpy arrays
temperature = np.asarray(temperature_a)
wind_direction = np.asarray(wind_direction_a)
wind_speed = np.asarray(wind_speed_a)
lons = np.asarray(lons_a)
lats = np.asarray(lats_a)

# If you want to measure time, uncomment the next line
#start = t.time()

for x in range(1, 7):

    # Read the pressures as python arrays
    pressure = np.asarray(pressure_a)

    # Plot the wind vectors divided in 6 pressure ranges, separated by color
    if (x == 1):
        print ("Plotting Pressure Range 1: (249-100 hPa)")
        pressure_index = np.where(( pressure >= 100 ) & ( pressure <= 249 ))[0]
        color = '#0000FF' # Blue
    elif (x == 2):
        print ("Plotting Range 2: (399-250 hPa)")
        pressure_index = np.where(( pressure >= 250 ) & ( pressure <= 399 ))[0]
        color = '#309AFF' # Light Blue
    elif (x == 3):
        print ("Plotting Range 3: (400-549 hPa)")
        pressure_index = np.where(( pressure >= 400 ) & ( pressure <= 549 ))[0]
        color = '#00FF00' # Green
    elif (x == 4):
        print ("Plotting Range 4: (699-550 hPa)")
        pressure_index = np.where(( pressure >= 550 ) & ( pressure <= 699 ))[0]
        color = '#FFFF00' # Yellow
    elif (x == 5):
        print ("Plotting Range 5: (849-700 hPa)")
        pressure_index = np.where(( pressure >= 700 ) & ( pressure <= 849 ))[0]
        color = '#FF0000' # Red
    elif (x == 6):
        print ("Plotting Range 6: (1000-850 hPa)")
        pressure_index = np.where(( pressure >= 850 ) & ( pressure <= 1000 ))[0]
        color = '#FF2FCD' # Violet

    # Create the variables lists (considerign only the given pressure range)
    pressure_b = []
    temperature_b = []
    wind_direction_b = []
    wind_speed_b = []
    lats_b = []
    lons_b = []

    # For each item, append the values to the respective variables
    for item in pressure_index:
        lons_b.append(lons_a[item])
        lats_b.append(lats_a[item])
        pressure_b.append(pressure_a[item])
        temperature_b.append(temperature_a[item])
        wind_direction_b.append(wind_direction_a[item])
        wind_speed_b.append(wind_speed_a[item])

    # Final variables for the given pressure range
    # Read the variables as numpy arrays
    pressure = np.asarray(pressure_b)
    temperature = np.asarray(temperature_b)
    wind_direction = np.asarray(wind_direction_b)
    wind_speed = np.asarray(wind_speed_b)
    lons = np.asarray(lons_b)
    lats = np.asarray(lats_b)

    # Calculating the u and v components using the wind_speed and wind direction
    # in order to plot the barbs. Reference:
    # https://earthscience.stackexchange.com/questions/11982/plotting-wind-barbs-in-python-no-u-v-component
    u = []
    v = []
    for item in range(lons.shape[0]):
        u.append(-(wind_speed[item]) * math.sin((math.pi / 180) * wind_direction[item]))
        v.append(-(wind_speed[item]) * math.cos((math.pi / 180) * wind_direction[item]))

    # Read the u and v components as numpy arrays
    u_comp = np.asarray(u)
    v_comp = np.asarray(v)

    # Create the Basemap
    bmap = Basemap(llcrnrlon=extent[0], llcrnrlat=extent[1], urcrnrlon=extent[2], urcrnrlat=extent[3], epsg=4326)
    x,y = bmap(lons, lats)
    # Make the barb plot
    bmap.barbs(x, y, u_comp, v_comp, length=7, pivot='middle', barbcolor=color)


# If you want to measure the time required to process, uncomment the next line
#print ('- Finished! Time required:', t.time() - start, 'seconds')

bmap.bluemarble()
# Draw parallels and meridians
bmap.drawparallels(np.arange(-90.0, 90.0, 5.0), linewidth=0.2, dashes=[5, 5], color='white', labels=[False,False,False,False], fmt='%g', labelstyle="+/-", xoffset=-0.80, yoffset=-1.00, size=7)
bmap.drawmeridians(np.arange(0.0, 360.0, 5.0), linewidth=0.2, dashes=[5, 5], color='white', labels=[False,False,False,False], fmt='%g', labelstyle="+/-", xoffset=-0.80, yoffset=-1.00, size=7)
bmap.readshapefile('content/Shapefiles/ne_10m_admin_1_states_provinces/ne_10m_admin_1_states_provinces','ne_10m_admin_1_states_provinces',linewidth=0.10,color='yellow')
# Add the countries shapefile
bmap.readshapefile('content/Shapefiles/ne_10m_admin_0_countries/ne_10m_admin_0_countries','ne_10m_admin_0_countries',linewidth=0.50,color='white')

# Plot the data
plot_g16(data_im, extent, product, variable, date_save, time_save)

# Insert the legend
plt.text(extent[0] + 0.5,extent[1] + 6.6,'249-100 hPa',  fontsize = 25,color='#0000FF')
plt.text(extent[0] + 0.5,extent[1] + 5.5,'399-250 hPa',  fontsize = 25,color='#309AFF')
plt.text(extent[0] + 0.5,extent[1] + 4.5,'400-549 hPa',  fontsize = 25,color='#00FF00')
plt.text(extent[0] + 0.5,extent[1] + 3.5,'699-550 hPa',  fontsize = 25,color='#FFFF00')
plt.text(extent[0] + 0.5,extent[1] + 2.5,'849-700 hPa',  fontsize = 25,color='#FF0000')
plt.text(extent[0] + 0.5,extent[1] + 1.5,'1000-850 hPa', fontsize = 25,color='#FF2FCD')

Title = "GOES-16 ABI CMI Band " + str(Band) + " " + Wavelenghts[int(Band)] + " " + Unit + " " + date + " " + time
plt.text(extent[0] + 0.5,extent[1] + 0.5,Title, fontsize = 25,color='white')



# Save the result
#plt.savefig('E:\\VLAB\\Python\\Output\\DMW Examples\\G16_DWMF_C' + str(Band) + '_' + date_save + time_save + '.png', dpi=DPI, pad_inches=0)
#plt.close()